In [1]:
%pip show -q selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


#### Uplash Image 다운로드

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import os
import time

# Chrome 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 창 안띄우기
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) #, options=options)

url = 'https://unsplash.com/s/photos/cat'
driver.get(url)

# JS 로딩 시간 대기
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
                              # ^= -> 해당하는 이라는 의미
img_tags = soup.select('img[src^="https://images.unsplash.com/photo"]') #=> image중에 해당 경로로 시작하는 이미지 가져오기
print(f'이미지 개수: {len(img_tags)}')

img_urls = [tag['src'] for tag in img_tags]

save_dir = 'unsplash_cat'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for idx, img_url in enumerate(img_urls, 1):
    res = requests.get(img_url)
    if res.ok:
        file_name = os.path.join(save_dir, f'cat_{idx}.jpg')
        with open(file_name, 'wb') as f:
            print(f'Saving... {file_name}')
            f.write(res.content)
    else:
        print(f'Failed to download {img_url}')

driver.quit()

#### Google Image 다운로드

In [ ]:
# 필요한 라이브러리 import
from selenium import webdriver  # 브라우저 자동화 도구
from selenium.webdriver.chrome.service import Service  # 크롬 드라이버 서비스 설정
from selenium.webdriver.common.by import By  # 요소 선택 방식
from selenium.webdriver.common.keys import Keys  # 키보드 입력 처리
from webdriver_manager.chrome import ChromeDriverManager  # 크롬 드라이버 자동 설치
from bs4 import BeautifulSoup  # HTML 파싱 라이브러리
import requests  # HTTP 요청 라이브러리
import os  # 파일/폴더 처리
import time  # 대기 처리

# 검색 키워드 및 이미지 저장 폴더명 설정
keyword = 'cat'  # 구글 이미지 검색 키워드
save_dir = 'google_img_cat'  # 이미지를 저장할 폴더 이름

# 크롬 브라우저 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저 창을 띄우지 않고 실행
options.add_argument('--no-sandbox')  # 샌드박스 모드 해제 (리눅스 서버 환경 대응)
options.add_argument('--disable-dev-shm-usage')  # shared memory 공간 부족 문제 해결용 옵션

# 크롬 드라이버 실행 (자동 설치 & 설정)
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), 
    options=options
)

# 구글 이미지 검색 URL 구성
search_url = f'https://www.google.com/search?tbm=isch&q={keyword}'
driver.get(search_url)  # 해당 URL로 접속

time.sleep(2)  # 페이지 로딩 대기 (2초)

# 스크롤을 여러번 내려서 더 많은 이미지 로딩
for _ in range(3):  # 3번 스크롤
    #                   TAG_NAME이 body인 element 찾기
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)  # END 키 입력 (맨 아래로 스크롤)
    time.sleep(2)  # 이미지 로딩 대기 (2초)

# 현재 페이지의 HTML 소스 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 모든 img 태그 추출
img_tags = soup.select('img')

print(f'img 태그 개수: {len(img_tags)}')  # 찾은 img 태그 개수 출력

# 이미지 src 속성 값 모으기
img_urls = []
for img in img_tags:
    src = img.get('src')  # 이미지 URL 추출
    if src and src.startswith('http'):  # http 로 시작하는 유효한 이미지 URL만 필터링
        img_urls.append(src)

print(f'수집 이미지 개수: {len(img_urls)}')  # 최종 이미지 URL 개수 출력

# 이미지 저장 폴더 없으면 새로 생성
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# 이미지 다운로드 및 저장
for idx, img_url in enumerate(img_urls[:10], 1):  # 이미지 URL 순서대로 반복
    try:
        res = requests.get(img_url, timeout=5)  # 이미지 요청 (최대 5초 대기)
        if res.ok:  # 정상 응답 시
            file_name = os.path.join(save_dir, f'{keyword}_{idx}.jpg')  # 파일 경로 및 이름 구성
            with open(file_name, 'wb') as f:  # 이미지 파일 쓰기 모드
                print(f'Saving... {file_name}')  # 저장 로그 출력
                f.write(res.content)  # 이미지 데이터 저장
    except Exception as e:  # 예외 처리 (타임아웃, 연결 오류 등)
        print(f'Failed to download {img_url} => {e}')  # 에러 로그 출력

driver.quit()  # 크롬 브라우저 종료



In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests # 이미지 파일 다운로드를 위한 HTTP 요청 라이브러리
import os       # 폴더 생성 및 파일 경로 관리를 위한 라이브러리
import time

# 사용자 입력 받기
keyword = input('검색할 키워드 입력: ')
target_count = int(input('다운로드할 이미지 개수 입력: '))

save_dir = f'google_img_{keyword}' # 검색어를 포함한 저장 폴더 이름 설정

# 크롬 설정 (브라우저 창을 띄우지 않는 옵션 등)
options = webdriver.ChromeOptions()
options.add_argument('--headless')           # 브라우저 창을 화면에 띄우지 않고 백그라운드에서 실행
options.add_argument('--no-sandbox')         # 리눅스 환경 등에서 보안 샌드박스 기능을 해제
options.add_argument('--disable-dev-shm-usage') # 메모리가 적은 환경에서 공유 메모리 사용 제한으로 인한 에러 방지

# 크롬 드라이버 실행 (옵션 적용)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 구글 이미지 검색 URL 설정 (tbm=isch는 이미지 검색 모드 의미)
search_url = f'https://www.google.com/search?tbm=isch&q={keyword}'
driver.get(search_url) # 설정한 URL로 이동
time.sleep(2)          # 페이지 초기 로딩 대기

# 이미지 충분히 로드될 때까지 스크롤 반복
while True:
    # 키보드의 END 키를 보내 페이지 맨 아래로 스크롤
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(2) # 스크롤 후 새로운 이미지가 로드될 때까지 대기
    
    html = driver.page_source             # 현재까지 로드된 페이지의 전체 HTML 소스 가져오기
    soup = BeautifulSoup(html, 'html.parser') # HTML 분석을 위해 BeautifulSoup 객체 생성
    img_tags = soup.select('img')         # 페이지 내의 모든 <img> 태그 선택
    
    print(f'현재 이미지 개수: {len(img_tags)}') # 진행 상황 파악을 위한 출력
    
    # 수집된 태그 수가 목표 개수(+여유분 10개)보다 많아지면 스크롤 중단
    if len(img_tags) >= target_count + 10:
        break

# src(이미지 주소) 속성값 추출
img_urls = []
for img in img_tags:
    src = img.get('src') # 이미지 태그의 소스 URL 가져오기
    # 주소가 존재하고, 실제 웹 주소(http)로 시작하는 경우만 리스트에 추가
    if src and src.startswith('http'):
        img_urls.append(src)

print(f'수집된 이미지 URL 개수: {len(img_urls)}')

# 저장 폴더가 컴퓨터에 없으면 새로 생성
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# 이미지 다운로드 반복문
for idx, img_url in enumerate(img_urls[:target_count], 1): # 수집된 URL 중 목표 개수만큼만 처리
    try:
        res = requests.get(img_url, timeout=5) # 이미지 URL에 접속하여 데이터 요청 (5초 타임아웃)
        if res.ok: # 요청이 성공(200 OK)했다면
            # 저장할 파일 경로 설정 (예: google_img_cat/cat_1.jpg)
            file_name = os.path.join(save_dir, f'{keyword}_{idx}.jpg')
            # 파일을 바이너리 쓰기(wb) 모드로 열어 이미지 데이터 저장
            with open(file_name, 'wb') as f:
                print(f'Saving... {file_name}') # 저장 알림 출력
                f.write(res.content)            # 이미지의 실제 데이터(content)를 파일에 기록
    except Exception as e:
        # 다운로드 중 에러(연결 끊김, 권한 등) 발생 시 에러 메시지 출력 후 다음 이미지로 진행
        print(f'Failed to download {img_url} => {e}')

driver.quit() # 모든 작업 완료 후 브라우저 종료

현재 이미지 개수: 253
수집된 이미지 URL 개수: 30
Saving... google_img_강아지\강아지_1.jpg
Saving... google_img_강아지\강아지_2.jpg
Saving... google_img_강아지\강아지_3.jpg
Saving... google_img_강아지\강아지_4.jpg
Saving... google_img_강아지\강아지_5.jpg
Saving... google_img_강아지\강아지_6.jpg
Saving... google_img_강아지\강아지_7.jpg
Saving... google_img_강아지\강아지_8.jpg
Saving... google_img_강아지\강아지_9.jpg
Saving... google_img_강아지\강아지_10.jpg
Saving... google_img_강아지\강아지_11.jpg
Saving... google_img_강아지\강아지_12.jpg
Saving... google_img_강아지\강아지_13.jpg
Saving... google_img_강아지\강아지_14.jpg
Saving... google_img_강아지\강아지_15.jpg
Saving... google_img_강아지\강아지_16.jpg
Saving... google_img_강아지\강아지_17.jpg
Saving... google_img_강아지\강아지_18.jpg
Saving... google_img_강아지\강아지_19.jpg
Saving... google_img_강아지\강아지_20.jpg
